In [15]:
%load_ext autoreload
%autoreload 2

%load_ext dotenv
%dotenv

In [34]:
# to reload
%dotenv

In [35]:
from google.cloud import bigquery

client = bigquery.Client()

In [60]:
p_sample = pd.read_excel("../projects/gain_tracker/tests/positions_sample.xlsx")

In [44]:
p_sample.dtypes

portfolio_id                    object
position_id                      int64
external_id                     object
id_type                         object
position_entry_date     datetime64[ns]
exchange                        object
quantity                         int64
status                          object
position_entry_price           float64
position_exit_date      datetime64[ns]
position_exit_price            float64
recognized_gain                float64
dtype: object

In [71]:
p_sample.loc[:, "recognized_gain"] = 1.2-1

In [76]:
p_sample.loc[:, "recognized_gain"] = 0.2

In [72]:
p_sample["recognized_gain"].values[0]

0.19999999999999996

In [77]:
job = client.load_table_from_dataframe(
    p_sample, "main-street-labs.gain_tracker_dev.positions"
)

In [107]:
# Perform a query.
QUERY = (
    'SELECT position_id, position_entry_price, position_exit_price FROM `main-street-labs.gain_tracker_dev.positions`'
)
p_sample_dl = client.query_and_wait(QUERY).to_dataframe().head(31)  # API request

### change to Decimals

In [47]:
from decimal import Decimal

In [49]:
p_sample.loc[:, "position_entry_price"] = p_sample["position_entry_price"].apply(
    lambda p: Decimal(str(round(p, 6))))

In [50]:
p_sample.loc[:, "position_exit_price"] = p_sample["position_exit_price"].apply(
    lambda p: Decimal(str(round(p, 6))))

In [52]:
p_sample.tail()

,portfolio_id,position_id,external_id,id_type,position_entry_date,exchange,quantity,status,position_entry_price,position_exit_date,position_exit_price,recognized_gain
26,Roth IRA,26,TSLA,TICKER,2024-01-25,PCX,2,OPEN_LONG,180.4,NaT,NaN,NaN
27,IRA,27,DQ,TICKER,2024-01-25,PCX,20,OPEN_LONG,19.5,NaT,NaN,NaN
28,IRA,28,CVS,TICKER,2024-01-25,PCX,8,OPEN_LONG,71.99,NaT,NaN,NaN
29,Roth IRA,29,BLDE,TICKER,2024-01-30,PCX,160,OPEN_LONG,3.0,NaT,NaN,NaN
30,IRA,30,GPS,TICKER,2024-01-31,PCX,20,OPEN_LONG,18.9,NaT,NaN,NaN


In [53]:
p_sample.dtypes

portfolio_id                    object
position_id                      int64
external_id                     object
id_type                         object
position_entry_date     datetime64[ns]
exchange                        object
quantity                         int64
status                          object
position_entry_price            object
position_exit_date      datetime64[ns]
position_exit_price             object
recognized_gain                float64
dtype: object

In [54]:
job = client.load_table_from_dataframe(
    p_sample, "main-street-labs.gain_tracker_dev.positions_dec"
)

In [56]:
# Perform a query.
QUERY = (
    'SELECT position_id, position_entry_price, position_exit_price FROM `main-street-labs.gain_tracker_dev.positions_dec`'
)
p_dec_sample_dl = client.query_and_wait(QUERY).to_dataframe()  # API request

In [108]:
p_sample_dl.tail()

,position_id,position_entry_price,position_exit_price
26,1,156.000,158.0000
27,0,210.130,470.0000
28,4,7.895,9.4000
29,5,7.895,9.2349
30,3,39.030,46.0700


In [98]:
1.3-1.1

0.19999999999999996

In [112]:
entry_price_dec = p_dec_sample_dl["position_entry_price"].apply(lambda p: p-Decimal("1.1"))

In [58]:
p_dec_sample_dl.tail()

,position_id,position_entry_price,position_exit_price
26,1,156.000000000,158.000000000
27,0,210.130000000,470.000000000
28,4,7.895000000,9.400000000
29,5,7.895000000,9.234900000
30,3,39.030000000,46.070000000


In [99]:
p_sample_dl.loc[0,"position_entry_price"]=1.3

In [109]:
entry_price_fl = p_sample_dl["position_entry_price"].apply(lambda p: p-1.1)

In [113]:
entry_price_dec.values[3]

Decimal('7.050000000')

In [111]:
entry_price_fl.values

array([ 52.0868,  89.1776, 277.9   ,   7.05  ,   6.3365,  19.4   ,
       163.9   ,  18.4   ,  70.89  ,  17.8   ,  44.4   , 163.7   ,
        16.    ,  16.02  ,   2.5   ,   2.12  ,  21.4   ,   7.61  ,
        13.2   , 217.5222,  18.9   , 213.9   ,  42.9   , 179.3   ,
         1.9   ,   2.12  , 154.9   , 209.03  ,   6.795 ,   6.795 ,
        37.93  ])

In [110]:
entry_price_fl.values[3]

7.050000000000001